# Keyword 검색시 - BERT 사용

In [1]:
# 로컬에서 BERT 사용할 경우 conda로 설치

!pip install -q -U "tensorflow-text==2.8.*"

     |████████████████████████████████| 4.9 MB 16.8 MB/s 
     |████████████████████████████████| 462 kB 34.1 MB/s 


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Bigquery 연동 패키지 import
from google.cloud import bigquery
from google.oauth2 import service_account

In [3]:
import glob
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from pandas.io import gbq

In [12]:
import re
import pandas as pd
import json
from pandas.io import gbq

In [4]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [40]:
credentials = service_account.Credentials.from_service_account_file('/content/drive/Shareddrives/BKMS/BKMS_FinalProject/recommendation_model/local_bigquery/dbmsproject-350608-bc1d3264916d.json')

In [129]:
client = bigquery.Client(project = 'dbmsproject-350608', credentials = credentials)
query = 'SELECT * FROM part1json.prac1'
df = client.query(query).to_dataframe()

In [99]:
len(df['title'])

72

In [130]:
#df2 = df.drop_duplicates(subset = ['title'])
df['title'] = df['title'].apply(lambda x: x.lower())
df['title'] = df['title'].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))

In [131]:
df.to_csv('practice.csv')

In [101]:
# Embedding 생성

preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1", trainable=True)

def get_bert_embeddings(text, preprocessor, encoder):

  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
  encoder_inputs = preprocessor(text_input)
  outputs = encoder(encoder_inputs)
  embedding_model = tf.keras.Model(text_input, outputs['pooled_output'])
  sentences = tf.constant([text])
  return embedding_model(sentences)

df['embeddings'] = df['title'].apply(lambda x: get_bert_embeddings(x, preprocessor, encoder))

In [108]:
df3 = pd.DataFrame(columns = ['title', 'year', 'embeddings'])

In [109]:
# Embedding값 확인
len(df['embeddings'])

72

In [96]:
df['embeddings'][0]

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[-0.9999952 ,  0.08767563, -0.9983914 , -0.04929075, -0.9887587 ,
         0.24698836, -0.9195187 , -0.8266282 ,  0.09789889, -0.09068467,
        -0.3372449 , -0.08456564, -0.07520106,  0.9999956 ,  0.10478592,
        -0.9473658 ,  0.14586918,  0.06156182, -0.9740967 ,  0.5976475 ,
         0.91572255, -0.01536728,  0.93368644, -0.28051174, -0.99977076,
         0.01216337, -0.999556  , -0.00157554,  0.9963304 ,  0.02028191,
        -0.05029644,  0.00812692, -0.9876165 , -0.9651693 ,  0.82195497,
         0.9999408 , -0.9537998 ,  0.00959275,  0.99720806, -0.99579835,
         0.99127424,  0.98080266, -0.99861383,  0.87759155, -0.998567  ,
        -0.21533714, -0.9891702 ,  0.9992789 ,  0.9544521 ,  0.9995659 ,
        -0.13092157, -0.10069051, -0.08414844, -0.80593896,  0.94850737,
         0.9727819 , -0.5938447 ,  0.14668104,  0.9989169 , -0.58427423,
         0.09900866,  0.6824307 , -0.9641355 ,  0.9572978 , -0.9832915 ,
 

In [90]:
type(df2['embeddings'][0].numpy())

numpy.ndarray

In [114]:
for i in range(len(df['embeddings'])):
  df['embeddings'][i] = df['embeddings'][i].numpy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [117]:
a = 'a'

In [121]:
b = []

In [116]:
df['embeddings'][0]

array([[-0.9999952 ,  0.08767563, -0.9983914 , -0.04929075, -0.9887587 ,
         0.24698836, -0.9195187 , -0.8266282 ,  0.09789889, -0.09068467,
        -0.3372449 , -0.08456564, -0.07520106,  0.9999956 ,  0.10478592,
        -0.9473658 ,  0.14586918,  0.06156182, -0.9740967 ,  0.5976475 ,
         0.91572255, -0.01536728,  0.93368644, -0.28051174, -0.99977076,
         0.01216337, -0.999556  , -0.00157554,  0.9963304 ,  0.02028191,
        -0.05029644,  0.00812692, -0.9876165 , -0.9651693 ,  0.82195497,
         0.9999408 , -0.9537998 ,  0.00959275,  0.99720806, -0.99579835,
         0.99127424,  0.98080266, -0.99861383,  0.87759155, -0.998567  ,
        -0.21533714, -0.9891702 ,  0.9992789 ,  0.9544521 ,  0.9995659 ,
        -0.13092157, -0.10069051, -0.08414844, -0.80593896,  0.94850737,
         0.9727819 , -0.5938447 ,  0.14668104,  0.9989169 , -0.58427423,
         0.09900866,  0.6824307 , -0.9641355 ,  0.9572978 , -0.9832915 ,
        -0.9999931 , -0.5108274 ,  0.9736905 ,  0.8

In [111]:
df3.head()

,title,year,embeddings


In [ ]:
client = bigquery.Client(project = 'dbmsproject-350608', credentials = credentials)

schema = [
    bigquery.SchemaField("title", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("embeddings", "ARRAY<INT64>", mode="REQUIRED"),
    bigquery.SchemaField("year", "INTEGER", mode="REQUIRED"),
    
]

table = bigquery.Table('dbmsproject-350608.part1json.embeddata2', schema=schema)
table = client.create_table(table)

In [42]:
df2.to_gbq(destination_table='part1json.embeddata', credentials = credentials,
		project_id='dbmsproject-350608', # Project id
		if_exists='replace')

1it [00:06,  6.55s/it]


In [43]:
client = bigquery.Client(project = 'dbmsproject-350608', credentials = credentials)
query = 'SELECT * FROM part1json.embeddata'
df_test = client.query(query).to_dataframe()

In [46]:
df_test['embeddings'][0]

'tf.Tensor(\n[[-0.9999952   0.08767563 -0.9983914  -0.04929075 -0.9887587   0.24698836\n  -0.9195187  -0.8266282   0.09789889 -0.09068467 -0.3372449  -0.08456564\n  -0.07520106  0.9999956   0.10478592 -0.9473658   0.14586918  0.06156182\n  -0.9740967   0.5976475   0.91572255 -0.01536728  0.93368644 -0.28051174\n  -0.99977076  0.01216337 -0.999556   -0.00157554  0.9963304   0.02028191\n  -0.05029644  0.00812692 -0.9876165  -0.9651693   0.82195497  0.9999408\n  -0.9537998   0.00959275  0.99720806 -0.99579835  0.99127424  0.98080266\n  -0.99861383  0.87759155 -0.998567   -0.21533714 -0.9891702   0.9992789\n   0.9544521   0.9995659  -0.13092157 -0.10069051 -0.08414844 -0.80593896\n   0.94850737  0.9727819  -0.5938447   0.14668104  0.9989169  -0.58427423\n   0.09900866  0.6824307  -0.9641355   0.9572978  -0.9832915  -0.9999931\n  -0.5108274   0.9736905   0.8894017   0.9941112   0.99884546  0.12141977\n  -0.9945616   0.02605322  0.813423   -0.99698055 -0.98995054  0.1230666\n   0.7884842   0

In [ ]:
tf.convert_to_tensor(
    value, dtype=None, dtype_hint=None, name=None
)

In [35]:
# 실행시 input값 입력 -> 추천 결과 확인 가능, 풀 네임이 아닌 키워드 입력으로도 가능

def preprocess_text():
  text = input()
  text = text.lower()
  text = re.sub('[^A-Za-z0-9]+', ' ', text)
  return text
  
query_text = preprocess_text()
query_encoding = get_bert_embeddings(query_text, preprocessor, encoder)

df2['similarity_score'] = df2['embeddings'].apply(lambda x: cosine_similarity(x, query_encoding)[0][0])
df_results = df2.sort_values(by=['similarity_score'], ascending=False)
df_results[['year', 'title', 'similarity_score']].head()

Timing yield estimation


,year,title,similarity_score
37,2008,fluctuations,0.967264
24,2005,timing yield estimation using statistical stat...,0.958424
30,2007,300,0.955083
41,2008,fgdc,0.953272
49,2009,fault,0.952051


In [19]:
# 테이블 생성 쿼리

client = bigquery.Client(project = 'dbmsproject-350608', credentials = credentials)

schema = [
    bigquery.SchemaField("title", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("similarity_score", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("year", "INTEGER", mode="REQUIRED"),
]

table = bigquery.Table('dbmsproject-350608.part1json.embed3', schema=schema)
table = client.create_table(table)

In [34]:
# 함수로 실행 -> DB로 보내기

def preprocess_text(title):
  text = title
  text = text.lower()
  text = re.sub('[^A-Za-z0-9]+', ' ', text)
  return text
  
def recommendation(title):
  query_text = preprocess_text(title)
  query_encoding = get_bert_embeddings(query_text, preprocessor, encoder)
  df2['similarity_score'] = df2['embeddings'].apply(lambda x: cosine_similarity(x, query_encoding)[0][0])
  df_s = df2.sort_values(by=['similarity_score'], ascending=False)
  df_results = df_s[['year', 'title', 'similarity_score']]
  df_results.to_gbq(destination_table='part1json.embed3', credentials = credentials,
		project_id='dbmsproject-350608', # Project id
		if_exists='replace')

In [21]:
recommendation('3gio')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
1it [00:04,  4.78s/it]


In [26]:
query = f"""
SELECT
  title, year, similarity_score,
FROM
  `dbmsproject-350608.part1json.embed3`
ORDER BY
  similarity_score DESC
LIMIT
  20
"""

In [126]:
# 쿼리 실행 결과

query_job = client.query(query)

In [127]:
# df로 변환

df = query_job.to_dataframe()

BadRequest: ignored

In [128]:
df

,year,title,embeddings
0,1957,foreword,"[[-0.9999952, 0.08767563, -0.9983914, -0.04929..."
1,1962,foreword,"[[-0.9999952, 0.08767563, -0.9983914, -0.04929..."
2,1966,foreword,"[[-0.9999952, 0.08767563, -0.9983914, -0.04929..."
3,1971,flocoder,"[[-0.9999996, 0.011695087, -0.9989764, 0.10357..."
4,1980,forward,"[[-0.9999989, 0.083291784, -0.9984166, -0.6802..."
...,...,...,...
67,2013,1,"[[-0.9999996, 0.14131233, -0.998664, -0.590672..."
68,2013,farewell,"[[-0.99999905, 0.101313576, -0.99929285, 0.409..."
69,2013,faircite,"[[-0.99999976, 0.07008639, -0.99912107, 0.3661..."
70,2013,foreword,"[[-0.9999952, 0.08767563, -0.9983914, -0.04929..."
